<a href="https://colab.research.google.com/github/MaxEV2003/Concentraci-n/blob/main/Dashboard%20Final%20Airbnb/Dashboard_mexico_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install streamlit

In [ ]:
%pip install plotly

In [ ]:
%pip install funpymodeling

In [ ]:
! npm install localtunnel

In [5]:
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [ ]:
dft=pd.read_csv("Mexico_listings_clean.csv")
dft.head(5)

In [22]:
%%writefile app.py
# Creamos el archivo de la APP en el interprete principal (Python)

# Importamos librerías
import streamlit as st
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from funpymodeling.exploratory import freq_tbl
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from PIL import Image
from scipy.optimize import curve_fit

# Definimos la instancia de streamlit
@st.cache_resource
def load_data():
    df = pd.read_csv("Mexico_listings_clean.csv", encoding='utf-8', index_col=0)
    df2= pd.read_csv("Mexico_listings_clean.csv", index_col="name")
    return df

# Cargamos los datos
data = load_data()
categoric_cols = data.select_dtypes(include=['object']).columns.tolist()  # Obtener columnas categóricas
numeric_cols = data.select_dtypes(include=['float', 'int']).columns.tolist()  # Obtener columnas numéricas

# Función para mostrar imágenes
def show_airbnb_image():
    img = Image.open("airbnb_logo.png")  # Asegúrate de tener esta imagen en el directorio
    st.image(img, caption='Airbnb Logo', use_column_width=True)

# Página principal
st.title('AIRBNB - México - ANÁLISIS DE DATOS')

# Sidebar - Selección de Página
# Insertar la imagen en la barra lateral (sidebar)
# URL of the image
image = "https://lauranoesta.com/wp-content/uploads/2019/09/airbnb-logo.png"
st.sidebar.image(image, use_column_width=True)
st.sidebar.title('MENU')
st.sidebar.header('Seleccione una Sección')
section = st.sidebar.selectbox(label='Sección', options=['Extracción de Características', 'Regresión Lineal Simple', 'Regresión Lineal Múltiple', 'Regresión No Lineal', 'Regresión Logística'])

# CONTENIDO - Extracción de Características
if section == 'Extracción de Características':
    st.subheader('Extracción de Características')

    st.sidebar.subheader('Tipo de Variables')
    vartype = st.sidebar.selectbox(label= 'Opción categorica', options=['Categorica'])

    if vartype == 'Categorica':
        st.text('Seleccione la variable y el tipo de gráfico')

        selected_var = st.selectbox(label='Variable', options=categoric_cols)
        selected_graph = st.selectbox(label='Gráfico', options=['Line Plot', 'Bar Plot', 'Scatter Plot', 'Pie Plot'])

        # Análisis univariado para variable categórica
        table = freq_tbl(data[selected_var])
        table1 = table.drop(['percentage', 'cumulative_perc'], axis=1)
        filtro = table1[table1['frequency'] > 1]
        filtro_index = filtro.set_index(str(selected_var))

        # Checkbox para mostrar tabla de frecuencias
        check_box = st.sidebar.checkbox(label="Mostrar Tabla de Frecuencias")
        if check_box:
            st.write(filtro_index)

        # Gráficos
        if selected_graph == 'Line Plot':
            figureEC = px.line(filtro_index, x=filtro_index.index, y='frequency', title='Line Plot', width=1600, height=600)
            st.plotly_chart(figureEC)

        elif selected_graph == 'Bar Plot':
            figureEC = px.bar(filtro_index, x=filtro_index.index, y='frequency', title='Bar Plot')
            st.plotly_chart(figureEC)

        elif selected_graph == 'Scatter Plot':
            figureEC = px.scatter(filtro_index, x=filtro_index.index, y='frequency', title="Scatter Plot")
            st.plotly_chart(figureEC)

        elif selected_graph == 'Pie Plot':
            figureEC = px.pie(filtro_index, names=filtro_index.index, values='frequency', title='Pie Plot', width=1600, height=600)
            st.plotly_chart(figureEC)

# CONTENIDO - Regresión Lineal Simple
if section == 'Regresión Lineal Simple':
    st.subheader('Regresión Lineal Simple')

    # Lógica para regresión lineal simple
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    if check_box:
        st.write(filtro_index)
        st.write(df)

    x_selected = st.sidebar.selectbox(label="(x) Variable independiente", options=numeric_cols)
    y_selected = st.sidebar.selectbox(label="(y) Variable dependiente", options=numeric_cols)

    # Modelo y resultados
    X = data[x_selected].values.reshape(-1, 1)
    y = data[y_selected].values
    model = LinearRegression()
    model.fit(X, y)
    coef_Deter = model.score(X, y)
    coef_Correl = np.sqrt(coef_Deter)

    # Predecir los valores de y para la línea de tendencia
    y_pred = model.predict(X)

    # Crear gráfico de dispersión con la línea de tendencia
    figure1 = px.scatter(data_frame=data, x=x_selected, y=y_selected,
                         title='Regresión lineal', color_discrete_sequence=["pink"])

    # Agregar línea de tendencia
    figure1.add_scatter(x=data[x_selected], y=y_pred, mode='lines', name='Línea de tendencia', line=dict(color='red', width=2))

    # Mostrar gráfico
    st.plotly_chart(figure1)

    st.markdown(f"<p style='font-size: 20px; color: white;'> Coeficiente de correlación (R): {coef_Correl:.4f}</p>", unsafe_allow_html=True)


# CONTENIDO - Regresión Lineal Múltiple
if section == 'Regresión Lineal Múltiple':
    st.subheader('Regresión Lineal Múltiple')
    # Lógica para regresión lineal múltiple
    # Seleccionar la variable dependiente (y) y las independientes (x)
    var_dep = st.sidebar.selectbox(label="Variable dependiente (x)", options=numeric_cols)  # Cambia numeric_cols por tus columnas
    vars_indep = st.sidebar.multiselect(label="Variables independientes (y)", options=numeric_cols)

    if vars_indep and var_dep:
        # Declaramos las variables dependientes e independientes para la regresión lineal
        Vars_Indep = data[vars_indep]
        Var_Dep = data[var_dep]

        # Definimos y ajustamos el modelo de regresión lineal
        model = LinearRegression()
        model.fit(X=Vars_Indep, y=Var_Dep)

        # Predecimos los valores usando el modelo ajustado
        y_pred = model.predict(X=Vars_Indep)

        # Insertamos las predicciones en el DataFrame
        data['Predicciones'] = y_pred

        # Visualizamos los gráficos comparativos
        for var in vars_indep:
            # Gráfico de dispersión de la variable independiente vs total real
            figure = px.scatter(data, x=var, y=var_dep, title=f'Relación entre {var} y {var_dep}',
                                labels={var: var, var_dep: 'Total'},
                                color_discrete_sequence=["orange"], opacity=0.5)

            # Agregar las predicciones como puntos
            figure.add_scatter(x=data[var], y=data['Predicciones'], mode='markers', name='Predicciones',
                               marker=dict(color='purple', size=10, symbol='star', line=dict(width=2, color='white')), opacity=0.8)

            # Mostrar el gráfico
            st.plotly_chart(figure)

        # Coeficiente de determinación (R²) y coeficiente de correlación
        coef_Deter = model.score(Vars_Indep, Var_Dep)
        coef_Correl = np.sqrt(coef_Deter)

        st.markdown(f"<p style='font-size: 20px; color: white;'> Coeficiente de correlación (R): {coef_Correl:.4f}</p>", unsafe_allow_html=True)
        st.markdown(f"<p style='font-size: 20px; color: white;'> Coeficiente de determinación (R²): {coef_Deter:.4f}</p>", unsafe_allow_html=True)
    else:
        st.warning("Selecciona una variable independiente y una variable dependiente.")

     # Definición de las funciones no lineales
def funcion_cuadratica(x, a, b, c):
    return a * x**2 + b * x + c

def funcion_exponencial(x, a, b, c):
    return a * np.exp(b * x) + c


def funcion_valor_absoluto(x, a, b, c):
    return a * np.abs(x) + b * x + c


def funcion_logaritmica(x, a, b):
    return a * np.log(x) + b



if section == 'Regresión No Lineal':
    st.subheader('Regresión No Lineal')

    # Selección de variables
    x_selected1 = st.sidebar.selectbox(label="Variable independiente (y)", options=numeric_cols)
    y_selected1 = st.sidebar.selectbox(label="Variable dependiente (x)", options=numeric_cols)

    # Selección de función de ajuste
    funcion_selected = st.sidebar.selectbox(
        "Selecciona la función de ajuste",
        options=["Función cuadrática", "Función exponencial",
                "Función Valor absoluto",
                 "Función logaritmica"]
    )

    # Diccionario de funciones
    funciones = {
        "Función cuadrática": funcion_cuadratica,
        "Función exponencial": funcion_exponencial,
        "Función Valor absoluto": funcion_valor_absoluto,
        "Función logaritmica": funcion_logaritmica,
    }

    # Obtener los datos seleccionados del DataFrame original
    x1 = data[x_selected1].values
    y1 = data[y_selected1].values

    # Inicializar y_pred1
    y_pred1 = None

    try:
        # Ajustar la función seleccionada a los datos
        popt, _ = curve_fit(funciones[funcion_selected], x1, y1)

        # Predecir los valores ajustados
        y_pred1 = funciones[funcion_selected](x1, *popt)

        # Crear el gráfico de dispersión y la curva ajustada
        fig = px.scatter(
            x=x1, y=y1, title=f'Ajuste de {funcion_selected}',
            labels={'x': x_selected1, 'y': y_selected1},
            template='plotly_white'
        )

        # Agregar la línea de ajuste
        fig.add_scatter(
            x=x1, y=y_pred1, mode='lines',
            name=f'Ajuste {funcion_selected}', line=dict(color='green', width=2)
        )

        # Mostrar el gráfico en Streamlit
        st.plotly_chart(fig)

    except Exception as e:
        st.error(f"Ocurrió un error al ajustar la función: {e}")

    # Verificar si y_pred1 fue definido antes de calcular la correlación
    if y_pred1 is not None:
        # Calcular el coeficiente de correlación
        correlacion1 = np.corrcoef(y1, y_pred1)[0, 1]

        # Mostrar el coeficiente de correlación
        st.markdown(
            f"<p style='font-size: 20px; color: grey;'> Coeficiente de correlación (R): {correlacion1:.4f}</p>",
            unsafe_allow_html=True
        )
    else:
        st.markdown(
            "<p style='font-size: 20px; color: grey;'> No se pudo calcular el coeficiente de correlación debido a un error en el ajuste.</p>",
            unsafe_allow_html=True
        )

# CONTENIDO - Regresión Logística
if section == 'Regresión Logística':
    st.subheader('Regresión Logística')

    # Selección de variables independientes y dependiente
    numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()  # Asegúrate de que 'data' esté definido
    Vars_Indep = st.sidebar.multiselect(label="Variables Independientes (X)", options=numeric_cols)
    Var_Dep = st.sidebar.selectbox(label="Variable Dependiente (Y)", options=data.columns)

    if len(Vars_Indep) > 0 and Var_Dep:
        # Variables independientes y dependiente
        X = data[Vars_Indep]
        y = data[Var_Dep]

        # Convertir y a tipo categórico si es necesario
        y = pd.Series(y)
        if y.dtype.kind in 'fi':  # Si 'y' es continuo
            y = pd.cut(y, bins=[-np.inf, 0.5, 1.5, np.inf], labels=['Bajo', 'Medio', 'Alto'])

        y = y.astype('category')

        # Verificar la cantidad de clases en y
        unique_classes = y.value_counts()
        st.write("Distribución de Clases en y:")
        st.write(unique_classes)

        # Asegúrate de que haya al menos dos clases
        if len(unique_classes) < 2:
            st.write("Error: La variable dependiente solo tiene una clase. Ajusta tus datos.")
        else:
            # Eliminar filas con valores faltantes para evitar problemas en el modelo
            df_model = pd.concat([X, y], axis=1).dropna()
            X = df_model[Vars_Indep].values
            y = df_model[Var_Dep].values.ravel()  # Asegúrate de que y sea un vector unidimensional

            # Verificar si las dimensiones son consistentes
            if len(X) > 0 and len(y) > 0 and X.shape[0] == y.shape[0]:
                # Dividir los datos en conjuntos de entrenamiento y prueba
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

                # Escalar solo las variables independientes
                escalar = StandardScaler()
                X_train = escalar.fit_transform(X_train)
                X_test = escalar.transform(X_test)

                # Definir y entrenar el modelo de regresión logística
                algoritmo = LogisticRegression()
                algoritmo.fit(X_train, y_train)

                # Realizar predicciones
                y_pred = algoritmo.predict(X_test)

                # Matriz de confusión
                matriz = confusion_matrix(y_test, y_pred)
                st.write('Matriz de Confusión:')
                st.write(matriz)

                # Calcular y mostrar métricas del modelo
                metrics = {
                    'Precisión (label=Yes)': precision_score(y_test, y_pred, pos_label='yes', average='macro'),
                    'Sensibilidad (label=Yes)': recall_score(y_test, y_pred, pos_label='yes', average='macro'),
                    'Exactitud del modelo': accuracy_score(y_test, y_pred),
                    'Precisión (label=No)': precision_score(y_test, y_pred, pos_label='no', average='macro'),
                    'Sensibilidad (label=No)': recall_score(y_test, y_pred, pos_label='no', average='macro')
                }

                for metric_name, metric_value in metrics.items():
                    st.write(f'{metric_name}: {metric_value:.4f}')

                # Valores de la matriz de confusión
                TN, FP, FN, TP = matriz.ravel()

                # Datos para la gráfica de pastel
                labels = ['Verdaderos Negativos', 'Falsos Positivos', 'Falsos Negativos', 'Verdaderos Positivos']
                sizes = [TN, FP, FN, TP]
                colors = ['lightblue', 'lightcoral', 'lightpink', 'lightgreen']
                explode = (0.1, 0.1, 0.1, 0.1)  # Destacar todas las porciones

            # Crear la gráfica de pastel
            fig, ax = plt.subplots()
            ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
            ax.axis('equal')  # Igualar el aspecto de la gráfica

            # Mostrar la gráfica en Streamlit
            st.pyplot(fig)

    else:
            st.write("Las variables seleccionadas no tienen el mismo número de filas. Revisa los datos.")
else:
        st.write("Por favor, selecciona al menos una variable independiente y una variable dependiente.")


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

104.196.160.232



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.160.232:8501

your url is: https://seven-planes-wait.loca.lt
